In [131]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import OrderedDict
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.decomposition import PCA
pd.set_option('display.max_columns', None)
cmaps = OrderedDict()

In [132]:
df = pd.read_csv(r'C:\Users\aaron\prem_league_analytics\data\clean_prem_players.csv')

In [133]:
df['player_position'] = df['player_positions'].str.split(',').str[0]
df['player_position'].unique()

array(['CB', '0', 'RB', 'CAM', 'LM', 'RW', 'RM', 'LW', 'LB', 'CM', 'CF',
       'ST', 'CDM', 'LWB', 'RWB', 'GK'], dtype=object)

In [134]:
df['player_position'] = df['player_position'].replace(to_replace =['RB', 'LB','LWB','RWB'], value='OB')
df['player_position'] = df['player_position'].replace(to_replace =['CAM','CDM', 'CM'], value='CM')
df['player_position'] = df['player_position'].replace(to_replace =['LM', 'RM', 'RW', 'LW'], value='W')
df['player_position'] = df['player_position'].replace(to_replace =['CF', 'ST'], value='ST')
df['player_position'] = df['player_position'].astype('str')

In [135]:
df['player_position'].unique()

array(['CB', '0', 'OB', 'CM', 'W', 'ST', 'GK'], dtype=object)

In [136]:
df['player_position_value'] = df['player_position']

In [137]:
df['player_position_value'] = df['player_position_value'].replace(to_replace =['GK'], value= 1)
df['player_position_value'] = df['player_position_value'].replace(to_replace =['CB'], value= 2)
df['player_position_value'] = df['player_position_value'].replace(to_replace =['OB'], value= 3)
df['player_position_value'] = df['player_position_value'].replace(to_replace =['CM'], value= 4)
df['player_position_value'] = df['player_position_value'].replace(to_replace =['W'], value= 5)
df['player_position_value'] = df['player_position_value'].replace(to_replace =['ST'], value= 6)

In [138]:
df = df.loc[df['year'] == '19/20']
df.head()

,Unnamed: 0,age,assists,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,bps,club,contract_valid_until,creativity,date,defending,defending_marking,defending_sliding_tackle,defending_standing_tackle,dribbling,fifa_name,fifa_name.1,games,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_speed,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goals,height,influence,international_reputation,joined_x,joined_y,left,match_name,mentality_aggression,mentality_composure,mentality_interceptions,mentality_penalties,mentality_positioning,mentality_vision,minutes,movement_acceleration,movement_agility,movement_balance,movement_reactions,movement_sprint_speed,mv,name,overall,pace,passing,physic,player_positions,player_positions.1,player_traits,position,potential,power_jumping,power_long_shots,power_shot_power,power_stamina,power_strength,preferred_foot,score,shooting,skill_ball_control,skill_curve,skill_dribbling,skill_fk_accuracy,skill_long_passing,skill_moves,team_jersey_number,team_position,threat,trans_name,transfermarkt_name,value_eur,wage_eur,weak_foot,web_name,year,player_position,player_position_value
4,4,31.0,0.0,52.0,54.0,84.0,75.0,57.0,102.0,Manchester City,2022.0,18.4,0,83.0,82.0,83.0,84.0,59.0,N. Otamendi,N. Otamendi,10.0,0.0,0.0,0.0,0.0,0.0,0.0,12,5,8,11,12,1.0,183.0,116.4,3.0,0,2015-08-20,0,0,91.0,79.0,83.0,45.0,48.0,53.0,873.0,55.0,58.0,58.0,78.0,57.0,0.0,Nicolás Otamendi,83.0,56.0,63.0,78.0,CB,CB,"Diver, Injury Prone, Long Passer (CPU AI Only)",Defender - Centre-Back,83.0,89.0,56.0,69.0,65.0,79.0,Right,0.0,57.0,70.0,50.0,51.0,39.0,72.0,2.0,30.0,SUB,204.0,0,Nicolás Otamendi,19000000,145000.0,3.0,Otamendi,19/20,CB,2
7,7,25.0,0.0,86.0,58.0,68.0,81.0,62.0,12.0,Manchester City,2025.0,3.1,"Aug 7, 2019",78.0,79.0,80.0,80.0,84.0,J. Cancelo,João Cancelo,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6,9,15,14,14,0.0,182.0,5.8,2.0,0,2019-08-07,Juventus,0,73.0,82.0,78.0,46.0,78.0,78.0,315.0,89.0,82.0,79.0,84.0,90.0,55000000.0,João Cancelo,84.0,90.0,80.0,71.0,RB,RB,Crowd Favourite,Defender - Right-Back,89.0,72.0,68.0,80.0,92.0,60.0,Right,0.0,65.0,84.0,82.0,85.0,69.0,75.0,4.0,27.0,SUB,0.0,0,João Cancelo,36500000,150000.0,3.0,Cancelo,19/20,OB,3
14,14,28.0,9.0,93.0,82.0,55.0,92.0,82.0,216.0,Manchester City,2023.0,380.3,0,61.0,68.0,51.0,58.0,86.0,K. De,K. De Bruyne,9.0,0.0,0.0,0.0,0.0,0.0,0.0,15,13,5,10,13,2.0,181.0,271.6,4.0,0,2015-08-30,0,0,76.0,91.0,61.0,79.0,88.0,94.0,697.0,77.0,78.0,76.0,91.0,76.0,0.0,Kevin De Bruyne,91.0,76.0,92.0,78.0,"CAM, CM","CAM, CM","Power Free-Kick, Avoids Using Weaker Foot, Div...",Midfielder - Attacking Midfield,91.0,63.0,90.0,91.0,89.0,74.0,Right,0.0,86.0,91.0,85.0,86.0,83.0,91.0,4.0,17.0,RCM,150.0,0,Kevin De Bruyne,90000000,370000.0,5.0,De Bruyne,19/20,CM,4
19,19,24.0,2.0,78.0,83.0,38.0,84.0,67.0,136.0,Manchester City,2023.0,128.4,0,45.0,47.0,47.0,53.0,89.0,R. Sterling,R. Sterling,11.0,0.0,0.0,0.0,0.0,0.0,0.0,15,12,12,15,9,9.0,170.0,216.6,3.0,0,2015-07-14,0,0,38.0,79.0,30.0,69.0,91.0,79.0,913.0,96.0,94.0,94.0,89.0,91.0,0.0,Raheem Sterling,88.0,93.0,78.0,57.0,"RW, LW","RW, LW","Selfish, Early Crosser",Forward - Left Winger,90.0,57.0,77.0,76.0,79.0,54.0,Right,0.0,79.0,88.0,75.0,89.0,63.0,69.0,4.0,7.0,LW,366.0,0,Raheem Sterling,73000000,255000.0,3.0,Sterling,19/20,W,5
24,24,23.0,0.0,83.0,81.0,72.0,80.0,85.0,0.0,Manchester City,2021.0,0.0,0,38.0,36.0,35.0,32.0,86.0,L. Sané,L. Sané,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8,12,9,9,14,0.0,184.0,0.0,2.0,0,2016-08-02,0,0,61.0,80.0,34.0,71.0,84.0,82.0,13.0,93.0,86.0,80.0,81.0,96.0,0.0,Leroy Sané,86.0,95.0,79.0,70.0,LW,LW,"Long Throw-in, Selfish, Early Crosser",Forward - Left Winger,92.0,64.0,78.0,87.0,78.0,70.0,Left,0.0,81.0,85.0,83.0,88.0,82.0,64.0,4.0,19.0,SUB,0.0,0,Leroy Sané,61000000,195000.0,3.0,0,19/20,W,5


In [139]:
features = ['attacking_crossing',
'attacking_finishing',
'attacking_heading_accuracy',
'attacking_short_passing',
'attacking_volleys',
'skill_dribbling',
'skill_curve',
'skill_fk_accuracy',
'skill_long_passing',
'skill_ball_control',
'movement_acceleration',
'movement_sprint_speed',
'movement_agility',
'movement_reactions',
'movement_balance',
'power_shot_power',
'power_jumping',
'power_stamina',
'power_strength',
'power_long_shots',
'mentality_aggression',
'mentality_interceptions',
'mentality_positioning',
'mentality_vision',
'mentality_penalties',
'mentality_composure',
'defending_marking',
'defending_standing_tackle',
'defending_sliding_tackle','creativity', 'threat', 'bps',
'overall','goals', 'assists']

In [140]:
df[features] = df[features].apply(pd.to_numeric)

In [141]:
train = train[train['player_position'] != '0']
train_feat = train[features]
train_feat.head()

,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,creativity,threat,bps,overall,goals,assists
0,52.0,53.0,86.0,64.0,58.0,43.0,50.0,39.0,66.0,60.0,74.0,76.0,71.0,80.0,62.0,69.0,90.0,78.0,82.0,56.0,85.0,86.0,49.0,45.0,44.0,0.0,87.0,86.0,88.0,0.0,0.0,0.0,84.0,1.0,0.0
1,52.0,53.0,86.0,64.0,56.0,41.0,50.0,39.0,66.0,60.0,68.0,75.0,71.0,79.0,62.0,69.0,90.0,78.0,82.0,56.0,85.0,86.0,49.0,45.0,44.0,76.0,83.0,86.0,86.0,0.0,0.0,0.0,85.0,1.0,3.0
2,52.0,53.0,83.0,71.0,56.0,41.0,50.0,39.0,66.0,64.0,62.0,64.0,69.0,78.0,62.0,69.0,86.0,76.0,80.0,56.0,90.0,85.0,49.0,45.0,44.0,84.0,81.0,83.0,84.0,0.0,0.0,0.0,83.0,5.0,0.0
3,52.0,54.0,85.0,75.0,57.0,49.0,50.0,39.0,72.0,70.0,59.0,61.0,64.0,79.0,62.0,69.0,92.0,67.0,80.0,56.0,91.0,86.0,51.0,53.0,45.0,84.0,85.0,87.0,86.0,0.0,0.0,0.0,86.0,1.0,1.0
10,90.0,82.0,53.0,88.0,82.0,85.0,78.0,77.0,82.0,86.0,76.0,78.0,78.0,87.0,75.0,84.0,65.0,85.0,73.0,85.0,68.0,52.0,83.0,88.0,77.0,0.0,30.0,39.0,40.0,0.0,0.0,0.0,86.0,17.0,15.0


In [142]:
#pca = PCA(.95)

In [143]:
#pca.fit(train_feat)

In [144]:
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['player_position']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [145]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
principalDf.head()

,principal component 1,principal component 2
0,-0.964405,-3.007560
1,-4.279087,-0.913389
2,-7.903763,2.838198
3,-5.393422,5.582215
4,-4.245149,2.724936


In [151]:
x = principalDf
y = df.loc[:,['player_position_value']].values
y = y.astype('int')
y = y.reshape(-1, 1)
y = column_or_1d(y, warn=True)

     principal component 1  principal component 2
0                -0.964405              -3.007560
1                -4.279087              -0.913389
2                -7.903763               2.838198
3                -5.393422               5.582215
4                -4.245149               2.724936
..                     ...                    ...
407              10.372858               1.739484
408              10.377617               1.604714
409               9.633348               1.537516
410              10.319229               2.189139
411              11.403987               1.774420

[412 rows x 2 columns]
[[2]
 [3]
 [4]
 [5]
 [5]
 [2]
 [2]
 [3]
 [4]
 [4]
 [6]
 [3]
 [4]
 [4]
 [2]
 [4]
 [4]
 [5]
 [6]
 [3]
 [4]
 [2]
 [3]
 [3]
 [4]
 [4]
 [4]
 [4]
 [2]
 [3]
 [3]
 [2]
 [5]
 [6]
 [2]
 [3]
 [2]
 [4]
 [5]
 [2]
 [3]
 [4]
 [3]
 [3]
 [4]
 [6]
 [6]
 [6]
 [6]
 [2]
 [3]
 [3]
 [4]
 [4]
 [5]
 [6]
 [6]
 [2]
 [3]
 [4]
 [5]
 [6]
 [6]
 [2]
 [3]
 [5]
 [6]
 [2]
 [4]
 [6]
 [6]
 [2]
 [4]
 [3]
 [4]
 

In [152]:
clf = NearestCentroid()
clf.fit(X, y)
NearestCentroid(metric='euclidean', shrink_threshold=None)
print(clf.predict([[-0.8, -1]]))

[3]


C:\Users\aaron\Anaconda3\envs\datascience\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [90]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])
clf = NearestCentroid()
clf.fit(X, y)
print(clf.predict([[-0.8, -1]]))

[1]
